In [1]:
library(tibble)
library(tidyverse)
library(lme4)
library(data.table)

── Attaching packages ──────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1
✔ purrr   0.3.4     

── Conflicts ─────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack



Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




In [2]:
all = fread('scores/variables.csv', sep='\t', header=T)
all <- all %>% mutate_if(is.character, list(~na_if(.,""))) 

In [3]:
raw = as_tibble(all) %>%
    mutate(seller_pictures=ifelse(is.na(seller_pictures), 0, 1),
         category_=as.factor(c(rep('elec', 7207), rep('bag', 14426-7206), rep('makeup', 26340-14426))))

In [4]:
## spec 1
model = glm(offered ~ seller_pictures + female + seller_level + seller_verification_email + seller_verification_phone + seller_verification_fb + product_price + product_images + log(product_title_length) + category_, family=binomial, data=raw)
summary(model)
logLik(model)


Call:
glm(formula = offered ~ seller_pictures + female + seller_level + 
    seller_verification_email + seller_verification_phone + seller_verification_fb + 
    product_price + product_images + log(product_title_length) + 
    category_, family = binomial, data = raw)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-1.432  -1.130  -1.023   1.216   1.434  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -3.564e-01  8.872e-02  -4.017 5.90e-05 ***
seller_pictures            1.095e-01  3.672e-02   2.982  0.00286 ** 
female                    -8.551e-02  2.666e-02  -3.208  0.00134 ** 
seller_level              -1.498e-02  5.173e-03  -2.896  0.00379 ** 
seller_verification_email -1.545e-01  3.175e-02  -4.865 1.15e-06 ***
seller_verification_phone  1.431e-01  6.329e-02   2.260  0.02381 *  
seller_verification_fb     8.787e-03  2.664e-02   0.330  0.74151    
product_price             -1.286e-09  1.905e-08  -0.067  

'log Lik.' -18135.52 (df=12)

In [5]:
## spec 2
dat <- raw %>% 
    filter(seller_pictures==1) %>%
    mutate(portrait=ifelse(num_faces>0, 1, 0))
print(dim(dat))
model = glm(offered ~ quality + portrait + female + seller_level + seller_verification_email + seller_verification_phone + seller_verification_fb + product_price + product_images + log(product_title_length) + category_, family=binomial, data=dat)
summary(model)
logLik(model)

[1] 22460    28



Call:
glm(formula = offered ~ quality + portrait + female + seller_level + 
    seller_verification_email + seller_verification_phone + seller_verification_fb + 
    product_price + product_images + log(product_title_length) + 
    category_, family = binomial, data = dat)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-1.423  -1.133  -1.026   1.211   1.413  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -4.492e-01  1.376e-01  -3.265 0.001095 ** 
quality                    6.535e-02  1.464e-01   0.446 0.655258    
portrait                   2.951e-02  2.843e-02   1.038 0.299198    
female                    -1.024e-01  2.932e-02  -3.492 0.000479 ***
seller_level              -1.743e-02  5.546e-03  -3.143 0.001670 ** 
seller_verification_email -1.720e-01  3.515e-02  -4.892 9.98e-07 ***
seller_verification_phone  2.090e-01  6.990e-02   2.991 0.002785 ** 
seller_verification_fb     1.511e-02  2.797e-02   0.54

'log Lik.' -15455.3 (df=13)

In [6]:
## spec 3
dat <- raw %>% 
    filter(num_faces==1)
model = glm(offered ~ quality + beauty + face_proximity + smile + female + seller_level + seller_verification_email + seller_verification_phone + seller_verification_fb + product_price + product_images + log(product_title_length) + category_, family=binomial, data=dat)
summary(model)
logLik(model)


Call:
glm(formula = offered ~ quality + beauty + face_proximity + smile + 
    female + seller_level + seller_verification_email + seller_verification_phone + 
    seller_verification_fb + product_price + product_images + 
    log(product_title_length) + category_, family = binomial, 
    data = dat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6161  -1.1233  -0.9808   1.2051   1.5034  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                1.652e-02  2.818e-01   0.059 0.953256    
quality                   -1.827e-01  2.798e-01  -0.653 0.513921    
beauty                    -4.985e-02  3.932e-02  -1.268 0.204819    
face_proximity             2.228e-01  1.389e-01   1.604 0.108674    
smile                     -2.066e-01  4.492e-02  -4.599 4.25e-06 ***
female                    -1.469e-01  4.749e-02  -3.093 0.001983 ** 
seller_level              -2.064e-02  9.396e-03  -2.196 0.028071 *  
seller_verificat

'log Lik.' -5710.203 (df=15)

In [7]:
## spec 4
dat <- raw %>% 
    filter(num_faces==1)
model = glm(offered ~ quality + beauty + face_proximity + smile + category_ : smile + female + seller_level + seller_verification_email + seller_verification_phone + seller_verification_fb + product_price + product_images + log(product_title_length) + category_, family=binomial, data=dat)
summary(model)
logLik(model)


Call:
glm(formula = offered ~ quality + beauty + face_proximity + smile + 
    category_:smile + female + seller_level + seller_verification_email + 
    seller_verification_phone + seller_verification_fb + product_price + 
    product_images + log(product_title_length) + category_, family = binomial, 
    data = dat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6014  -1.1236  -0.9777   1.2062   1.5086  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                2.125e-02  2.836e-01   0.075 0.940273    
quality                   -1.689e-01  2.801e-01  -0.603 0.546419    
beauty                    -5.023e-02  3.933e-02  -1.277 0.201497    
face_proximity             2.262e-01  1.389e-01   1.629 0.103396    
smile                     -2.346e-01  8.228e-02  -2.852 0.004350 ** 
female                    -1.442e-01  4.752e-02  -3.034 0.002415 ** 
seller_level              -2.105e-02  9.400e-03  -2.240 0.025117 * 

'log Lik.' -5708.872 (df=17)

In [8]:
## spec 5
dat <- raw %>% 
    filter(num_faces==1)
model = glm(offered ~ quality + beauty + face_proximity + Intensity + female + seller_level + seller_verification_email + seller_verification_phone + seller_verification_fb + product_price + product_images + log(product_title_length) + category_, family=binomial, data=dat)
summary(model)
logLik(model)


Call:
glm(formula = offered ~ quality + beauty + face_proximity + Intensity + 
    female + seller_level + seller_verification_email + seller_verification_phone + 
    seller_verification_fb + product_price + product_images + 
    log(product_title_length) + category_, family = binomial, 
    data = dat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6182  -1.1250  -0.9893   1.2057   1.5224  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                8.169e-02  2.844e-01   0.287 0.773926    
quality                   -2.012e-01  2.797e-01  -0.719 0.471887    
beauty                    -5.373e-02  3.931e-02  -1.367 0.171734    
face_proximity             2.580e-01  1.392e-01   1.854 0.063738 .  
Intensity                 -2.868e-01  8.635e-02  -3.322 0.000895 ***
female                    -1.631e-01  4.735e-02  -3.444 0.000574 ***
seller_level              -2.204e-02  9.392e-03  -2.346 0.018951 *  
seller_verif

'log Lik.' -5715.262 (df=15)

In [9]:
## spec 6
dat <- raw %>% 
    filter(num_faces==1)
model = glm(offered ~ quality + beauty + face_proximity + Intensity + I(Intensity^2) + female + seller_level + seller_verification_email + seller_verification_phone + seller_verification_fb + product_price + product_images + log(product_title_length) + category_, family=binomial, data=dat)
summary(model)
logLik(model)


Call:
glm(formula = offered ~ quality + beauty + face_proximity + Intensity + 
    I(Intensity^2) + female + seller_level + seller_verification_email + 
    seller_verification_phone + seller_verification_fb + product_price + 
    product_images + log(product_title_length) + category_, family = binomial, 
    data = dat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5966  -1.1272  -0.9865   1.2042   1.5430  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -2.894e-03  2.915e-01  -0.010 0.992078    
quality                   -2.086e-01  2.798e-01  -0.746 0.455922    
beauty                    -5.188e-02  3.934e-02  -1.319 0.187255    
face_proximity             2.688e-01  1.394e-01   1.928 0.053853 .  
Intensity                  1.477e-01  3.391e-01   0.436 0.663128    
I(Intensity^2)            -4.325e-01  3.266e-01  -1.324 0.185362    
female                    -1.591e-01  4.744e-02  -3.353 0.000798

'log Lik.' -5714.383 (df=16)

In [10]:
## spec 7
dat <- raw %>% 
    filter(num_faces==1)
model = glm(offered ~ quality + beauty + face_proximity + asymmetry + female + seller_level + seller_verification_email + seller_verification_phone + seller_verification_fb + product_price + product_images + log(product_title_length) + category_, family=binomial, data=dat)
summary(model)
logLik(model)


Call:
glm(formula = offered ~ quality + beauty + face_proximity + asymmetry + 
    female + seller_level + seller_verification_email + seller_verification_phone + 
    seller_verification_fb + product_price + product_images + 
    log(product_title_length) + category_, family = binomial, 
    data = dat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5749  -1.1293  -0.9993   1.2094   1.4748  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)               -6.706e-02  2.824e-01  -0.237 0.812287    
quality                   -2.075e-01  2.794e-01  -0.743 0.457647    
beauty                    -5.121e-02  3.928e-02  -1.304 0.192289    
face_proximity             2.247e-01  1.387e-01   1.620 0.105165    
asymmetry                  8.404e-03  4.527e-02   0.186 0.852748    
female                    -1.619e-01  4.734e-02  -3.421 0.000624 ***
seller_level              -2.141e-02  9.391e-03  -2.280 0.022606 *  
seller_verif

'log Lik.' -5720.775 (df=15)

In [11]:
## spec 8
dat <- raw %>% 
    filter(num_faces==1)
model = glm(offered ~ quality + beauty + face_proximity + Intensity + I(Intensity^2) + asymmetry + female + seller_level + seller_verification_email + seller_verification_phone + seller_verification_fb + product_price + product_images + log(product_title_length) + category_, family=binomial, data=dat)
summary(model)
logLik(model)


Call:
glm(formula = offered ~ quality + beauty + face_proximity + Intensity + 
    I(Intensity^2) + asymmetry + female + seller_level + seller_verification_email + 
    seller_verification_phone + seller_verification_fb + product_price + 
    product_images + log(product_title_length) + category_, family = binomial, 
    data = dat)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.5905  -1.1269  -0.9878   1.2039   1.5403  

Coefficients:
                            Estimate Std. Error z value Pr(>|z|)    
(Intercept)                8.395e-03  2.925e-01   0.029 0.977105    
quality                   -2.088e-01  2.798e-01  -0.746 0.455522    
beauty                    -5.189e-02  3.934e-02  -1.319 0.187092    
face_proximity             2.699e-01  1.394e-01   1.936 0.052886 .  
Intensity                  1.661e-01  3.416e-01   0.486 0.626738    
I(Intensity^2)            -4.559e-01  3.307e-01  -1.379 0.168011    
asymmetry                 -2.096e-02  4.628e-02  -0.

'log Lik.' -5714.281 (df=17)